# Imports

Brittany C. Haas and Melissa A. Hardy's jupyter notebook for automated collection of molecular descriptors and post-processing (i.e., Boltzmann average, min/max values, etc.).

**NOTE: Make sure to use the get_properties_environment file to set your conda environment.**

In [1]:
import os,re,sys,pickle,datetime,time,random,itertools,glob
import warnings
warnings.filterwarnings("ignore")
import openpyxl
import pandas as pd
from rdkit import Chem
import get_properties_functions as gp

D3 import failed


# Atom Inputs Dataframe

Portions of this section were adapted from code written Jordan P. Liles.

## Generate dataframe with atom numbers

### Use command line to prepare files

To create files: navigate to folder that contains all the log files you wish to analyze.

    module load openbabel
    obabel *.log -osdf -m  
    ls *.log > log_ids.txt
    cat *.sdf >> molecules.sdf

You will use the log_ids.txt and molecules.sdf files in the rest of 2.1.

### Define SMARTS substructure


Recommended to draw the common substructure (with general atoms) in Chemdraw and copy as SMILES (this will generate a SMARTS string)

More information about SMARTS and available characters here: https://www.daylight.com/dayhtml/doc/theory/theory.smarts.html


In [2]:
#this is the common smarts substructure for the molecules you will analyze
#you have to explicitly draw hydrogens into the SMARTS structure if you want to collect properties for hydrogen atoms
substructure = Chem.MolFromSmarts('[*]C(O[H])=O')

### Generate preliminary dataframe

In [3]:
#generate a list of molecules using RDkit
all_compounds = Chem.SDMolSupplier('molecules.sdf', removeHs=False) 
#molecules.sdf is generated with the instructions above
#it is a single sdf that contains the structures/atom numbers etc. for every molecule you will analyze

#uses RDKit to search for the substructure in each compound you will analyze
atoms = []
for molecule in all_compounds:
    if molecule is not None:
        submatch = molecule.GetSubstructMatches(substructure) #find substructure
        matchlist = list([item for sublist in submatch for item in sublist]) #list of zero-indexed atom numbers
        match_idx = [x+1 for x in matchlist] #this line changes from 0-indexed to 1-indexed (for Gaussian)
        atoms.append(match_idx) #append 1-indexed list to atoms (a list of lists)
        
#this loop extracts log names from log_ids and splits them to the desired format
filenames = open("log_ids.txt", "r") #generate this with instruction above
#it is a text file that contains the file name for every molecule you will analyze
list_of_filenames = [(line.strip()).split() for line in filenames] #list of the file names (each of which includes all conformers)
list_of_files = []
for filename in list_of_filenames:
    file = filename[0].split(".")
    list_of_files.append(file[0])
filenames.close()

#put the atom numbers for the substructure for each log file into a dataframe
prelim_df = pd.DataFrame(atoms) 
index=list_of_files
prelim_df.insert(0,column='log_name',value=list_of_files)
display(prelim_df)

,log_name,0,1,2,3,4
0,Ac1_clust-1,3,2,1,13,12
1,Ac1_clust-10,3,2,1,13,12
2,Ac1_clust-11,3,2,1,13,12
3,Ac1_clust-12,3,2,1,13,12
4,Ac1_clust-13,3,2,1,13,12
5,Ac1_clust-14,3,2,1,13,12
6,Ac1_clust-15,3,2,1,13,12
7,Ac1_clust-16,3,2,1,13,12
8,Ac1_clust-17,3,2,1,13,12
9,Ac1_clust-18,3,2,1,13,12


### Define column headers using GaussView

Using the preliminary dataframe displayed above, open one of your files and check the atom numbers. 

Assign labels to each column using the cell below. You will call these column headers when you select your properties. 

**User input required:**

In [4]:
atom_labels = {'log_name': 'log_name',
                0: 'C4',
                1: 'C1', 
                2: 'O3',
                3: 'H5',
                4: 'O2'}

### Generate labeled dataframe

**NOTE: it is very important you assign these correctly otherwise the properties you collect will be for the wrong atoms and not produce meaningful correlations.** 

We recommend checking the numbering/headers for at least two different compounds. 

Numbering for different conformers of the same compounds will likely be the same (but may not be for some symmetrical groups).

In [5]:
#rename columns using the user input above
atom_map_df = prelim_df.rename(columns=atom_labels)
display(atom_map_df)

#you can use this to clean up the table if you have more atoms in your substructure than you want to collect descriptors for
#atom_map_df = atom_map_df.drop(columns= ['C4', 'C1']) 
#display(atom_map_df.head())

df = atom_map_df #df is what properties will be appended to, this creates a copy so that you have the original preserved 

,log_name,C4,C1,O3,H5,O2
0,Ac1_clust-1,3,2,1,13,12
1,Ac1_clust-10,3,2,1,13,12
2,Ac1_clust-11,3,2,1,13,12
3,Ac1_clust-12,3,2,1,13,12
4,Ac1_clust-13,3,2,1,13,12
5,Ac1_clust-14,3,2,1,13,12
6,Ac1_clust-15,3,2,1,13,12
7,Ac1_clust-16,3,2,1,13,12
8,Ac1_clust-17,3,2,1,13,12
9,Ac1_clust-18,3,2,1,13,12


### Save atom map to Excel (if desired)

In [6]:
writer = pd.ExcelWriter('Ac1_to_Ac5_sample_atom_map.xlsx')
atom_map_df.to_excel(writer)
writer.save()

## Import a manually-generated atom mapping dataframe

If you need to manually generate the atom mapping dataframe, check out the atom_map_sample.xlsx to make sure you have the desired format. 

In [ ]:
atom_map_df = pd.read_excel('Ac1_to_Ac5_sample_atom_map.xlsx','Sheet1',index_col=0,header=0,engine='openpyxl')
display(atom_map_df.head())

# Define Properties to Collect

## Available property functions and how to call them: 

In [ ]:
#this box has functions to choose from
df = atom_map_df

#---------------GoodVibes Engergies---------------
#uses the GoodVibes 2021 Branch (Jupyter Notebook Compatible)
#calculates the quasi harmonic corrected G(T) and single point corrected G(T) as well as other thermodynamic properties
#inputs: dataframe, temperature
df = gp.get_goodvibes_e(df, 298.15)

#---------------Frontier Orbitals-----------------
#E(HOMO), E(LUMO), mu(chemical potential or negative of molecular electronegativity), eta(hardness/softness), omega(electrophilicity index)
df = gp.get_frontierorbs(df)

#---------------Polarizability--------------------
#Exact polarizability
df = gp.get_polarizability(df)

#---------------Dipole----------------------------
#Total dipole moment magnitude in Debye
df = gp.get_dipole(df)

#---------------Volume----------------------------
#Molar volume
#requires the Gaussian keyword = "volume" in the .com file
df = gp.get_volume(df)

#---------------SASA------------------------------
#Uses morfeus to calculat sovlent accessible surface area and the volume under the SASA
df = gp.get_SASA(df)

#---------------NBO-------------------------------
#natural charge from NBO
#requires the Gaussian keyword = "pop=nbo7" in the .com file
nbo_list = ["C1", "O2", "O3", "C4"]
df = gp.get_nbo(df, nbo_list) 

#---------------NMR-------------------------------
#isotropic NMR shift
#requires the Gaussian keyword = "nmr=giao" in the .com file
nmr_list = ["C1", "C4", "H5"]
df = gp.get_nmr(df, nmr_list) 

#---------------Distance--------------------------
#distance between 2 atoms
dist_list_of_lists = [["O2", "C1"], ["O3", "H5"], ["C4", "C1"]]
df = gp.get_distance(df, dist_list_of_lists) 

#---------------Angle-----------------------------
#angle between 3 atoms
angle_list_of_lists = [["O3", "C1", "O2"], ["C4", "C1", "O3"]]
df = gp.get_angles(df, angle_list_of_lists) 

#---------------Dihedral--------------------------
#dihedral angle between 4 atoms
dihedral_list_of_lists = [["O2", "C1", "O3", "H5"], ["C4", "C1", "O3", "H5"]]
df = gp.get_dihedral(df, dihedral_list_of_lists) 

#---------------Vbur Scan-------------------------
#uses morfeus to calculate the buried volume at a series of radii (including hydrogens)
#inputs: dataframe, list of atoms, start_radius, end_radius, and step_size
#if you only want a single radius, put the same value for start_radius and end_radius (keep step_size > 0)
vbur_list = ["C1", "C4"]
df = gp.get_vbur_scan(df, vbur_list, 2, 4, 0.5)
    
#---------------Sterimol morfeus------------------
#uses morfeus to calculate Sterimol L, B1, and B5 values
#NOTE: this is much faster than the corresponding DBSTEP function (recommendation: use as default/if you don't need Sterimol2Vec)
sterimol_list_of_lists = [["O3", "H5"], ["C1", "C4"]]
df = gp.get_sterimol_morfeus(df, sterimol_list_of_lists) 

#---------------Buried Sterimol-------------------
#uses morfeus to calculate Sterimol L, B1, and B5 values within a given sphere of radius r_buried
#atoms outside the sphere + 0.5 vdW radius are deleted and the Sterimol vectors are calculated
#for more information: https://kjelljorner.github.io/morfeus/sterimol.html
#inputs: dataframe, list of atom pairs, r_buried
sterimol_list_of_lists = [["C1", "C4"]]
df = gp.get_buried_sterimol(df, sterimol_list_of_lists, 5.5) 

#---------------Sterimol DBSTEP-------------------
#uses DBSTEP to calculate Sterimol L, B1, and B5 values
#default grid point spacing (0.05 Angstrom) is used (can use custom spacing or vdw radii in the get_properties_functions script)
#more info here: https://github.com/patonlab/DBSTEP
#NOTE: this takes longer than the morfeus function (recommendation: only use this if you need Sterimol2Vec)
sterimol_list_of_lists = [["O3", "H5"]]
df = gp.get_sterimol_dbstep(df, sterimol_list_of_lists) 

#---------------Sterimol2Vec----------------------
#uses DBSTEP to calculate Sterimol Bmin and Bmax values at intervals from 0 to end_radius, with a given step_size 
#default grid point spacing (0.05 Angstrom) is used (can use custom spacing or vdw radii in the get_properties_functions script)
#more info here: https://github.com/patonlab/DBSTEP
#inputs: dataframe, list of atom pairs, end_radius, and step_size
sterimol2vec_list_of_lists = [["C1", "C4"]]
df = gp.get_sterimol2vec(df, sterimol2vec_list_of_lists, 1, 1.0) 

#---------------Pyramidalization------------------
#uses morfeus to calculate pyramidalization based on the 3 atoms in closest proximity to the defined atom
#collects values based on two definitions of pyramidalization
#details on these values can be found here: https://kjelljorner.github.io/morfeus/pyramidalization.html
pyr_list = ["C1", "C4"]
df = gp.get_pyramidalization(df, pyr_list)

#---------------Plane Angle-----------------------
#plane angle between 2 planes (each defined by 3 atoms)
planeangle_list_of_lists = [["O2", "C1", "O3", "H5", "C1", "C4"], ["O2", "C1", "O3", "H5", "C1", "C4"]]
df = gp.get_planeangle(df, planeangle_list_of_lists) 

#---------------Time----------------------------------
#returns the total CPU time and total Wall time
#if used in summary df, will give the average (not Boltzmann average) in the Boltzmann average column
df = gp.get_time(df)

#---------------ChelpG----------------------------
#ChelpG ESP charge 
#requires the Gaussian keyword = "pop=chelpg" in the .com file
a_list = ['C1']
df = gp.get_chelpg(df, a_list) 

#---------------Hirshfeld-------------------------
#Hirshfeld charge, CM5 charge, Hirshfeld atom dipole
#requires the Gaussian keyword = "pop=hirshfeld" in the .com file
a_list = ['C1']
df = gp.get_hirshfeld(df, a_list) 

#---------------IR--------------------------------
#CAUTION: CANNOT ACCURATELY IDENTIFY ATOM STRETCHES IN SOME CASES (struggles if there is more than one stretch in the defined range)
#IR frequencies and intensities in a specific range (for specific atoms)
#requires the Gaussian keyword = "freq=noraman" in the .com file
#inputs: dataframe, atom1, atom2, frequency_min, frequency_max, intensity_min, intensity_max, threshold
#if you want to collect multiple IR frequencies, you will need to copy/paste this function for each stretch
#we recommend a threshold of 0.0 (may have to adjust)
df = gp.get_IR(df, "C1", "O2", 1700, 1900, 100, 800, 0.0)


pd.options.display.max_columns = None
display(df)

## Copy and modify available property functions above to customize

We recommend copying the entire cell above. You will need to change the atom number lists to match your desired column headers and delete (or comment out) any properites you don't want to collect.

In [7]:
#this box has functions to choose from
df = atom_map_df

#---------------GoodVibes Engergies---------------
#uses the GoodVibes 2021 Branch (Jupyter Notebook Compatible)
#calculates the quasi harmonic corrected G(T) and single point corrected G(T) as well as other thermodynamic properties
#inputs: dataframe, temperature
df = gp.get_goodvibes_e(df, 298.15)

#---------------Frontier Orbitals-----------------
#E(HOMO), E(LUMO), mu(chemical potential or negative of molecular electronegativity), eta(hardness/softness), omega(electrophilicity index)
df = gp.get_frontierorbs(df)

#---------------Vbur Scan-------------------------
#uses morfeus to calculate the buried volume at a series of radii (including hydrogens)
#inputs: dataframe, list of atoms, start_radius, end_radius, and step_size
#if you only want a single radius, put the same value for start_radius and end_radius (keep step_size > 0)
vbur_list = ["C1", "C4"]
df = gp.get_vbur_scan(df, vbur_list, 2, 4, 0.5)

pd.options.display.max_columns = None
display(df)



   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using vibrational scale factor 1.0 for B3LYP/6-31G(d,p) level of theory

   Using v

,log_name,C4,C1,O3,H5,O2,E_spc (Hartree),G(T)_spc(Hartree),H_spc(Hartree),T,T*S,T*qh_S,ZPE(Hartree),qh_G(T)_spc(Hartree),HOMO,LUMO,η,μ,ω,%Vbur_C1_2.0Å,%Vbur_C4_2.0Å,%Vbur_C1_2.5Å,%Vbur_C4_2.5Å,%Vbur_C1_3.0Å,%Vbur_C4_3.0Å,%Vbur_C1_3.5Å,%Vbur_C4_3.5Å,%Vbur_C1_4.0Å,%Vbur_C4_4.0Å
0,Ac1_clust-1,3,2,1,13,12,-543.543441,-543.291813,-543.235970,298.15,0.055843,0.054099,0.292507,-543.290069,-0.35353,0.03938,0.39291,-0.157075,0.03140,96.871772,97.691761,90.127295,84.967118,78.622244,71.062090,65.065924,60.080251,54.098520,49.841723
1,Ac1_clust-10,3,2,1,13,12,-543.537866,-543.292382,-543.230502,298.15,0.061880,0.057455,0.291621,-543.287958,-0.35342,0.04394,0.39736,-0.154740,0.03013,94.473140,97.536803,82.479815,85.004558,65.471357,71.097475,48.637835,59.546217,37.247455,48.395118
2,Ac1_clust-11,3,2,1,13,12,-543.539014,-543.290972,-543.231499,298.15,0.059473,0.056151,0.292038,-543.287650,-0.35148,0.04371,0.39519,-0.153885,0.02996,94.521565,98.276085,82.623063,87.275361,65.543057,75.093117,49.114214,64.315831,38.336773,53.410325
3,Ac1_clust-12,3,2,1,13,12,-543.540547,-543.294717,-543.233299,298.15,0.061417,0.057290,0.291455,-543.290589,-0.35482,0.04101,0.39583,-0.156905,0.03110,96.064695,97.136493,85.509181,82.992577,69.311494,67.682881,52.907193,54.992080,40.855627,43.393489
4,Ac1_clust-13,3,2,1,13,12,-543.538204,-543.290149,-543.230894,298.15,0.059255,0.055999,0.291839,-543.286894,-0.35491,0.03949,0.39440,-0.157710,0.03153,95.454545,97.520661,86.689348,85.657312,72.240018,72.403903,56.592434,60.887416,44.779576,49.495686
5,Ac1_clust-14,3,2,1,13,12,-543.539852,-543.290798,-543.232240,298.15,0.058558,0.055342,0.292347,-543.287582,-0.35479,0.04148,0.39627,-0.156655,0.03096,95.751550,97.184917,85.069671,82.927465,69.691411,67.819763,53.831998,55.741008,42.008954,44.794705
6,Ac1_clust-15,3,2,1,13,12,-543.539979,-543.290484,-543.232198,298.15,0.058285,0.055207,0.292559,-543.287406,-0.35389,0.03735,0.39124,-0.158270,0.03201,95.312500,97.255940,85.675218,85.641034,71.004358,73.308068,55.780609,62.845928,44.497936,51.945100
7,Ac1_clust-16,3,2,1,13,12,-543.537734,-543.289007,-543.230323,298.15,0.058684,0.055758,0.291958,-543.286081,-0.35407,0.03786,0.39193,-0.158105,0.03189,95.509427,97.433497,86.230303,85.658940,71.755810,73.658187,56.437523,63.144102,45.396779,52.466483
8,Ac1_clust-17,3,2,1,13,12,-543.540591,-543.291636,-543.232992,298.15,0.058644,0.055477,0.292298,-543.288469,-0.35321,0.03681,0.39002,-0.158200,0.03208,96.439179,97.294680,87.548834,83.817880,73.047341,69.557323,57.167234,58.266167,44.990224,47.251102
9,Ac1_clust-18,3,2,1,13,12,-543.542172,-543.294049,-543.234779,298.15,0.059269,0.055892,0.291974,-543.290671,-0.35763,0.04185,0.39948,-0.157890,0.03120,96.600594,97.427040,87.545579,83.977406,72.267022,69.966105,56.176039,58.693044,43.969958,47.363990


## Save collected properties to Excel

Helpful to save here in case the Notebook crashes or if you want to add more properties before post-processsing. Can be read in at 5.1.1.

In [8]:
writer = pd.ExcelWriter('All_Conformer_Properties_example.xlsx')
df.to_excel(writer)
writer.save()

# Post-processing

## User input for data processing

In [9]:
#for numerically named compounds, prefix is any text common to all BEFORE the number and suffix is common to all AFTER the number
#this is a template for our files that are all named "AcXXX_clust-X.log" or "AcXXX_conf-X.log"
prefix = "Ac" 
suffix = "_"

#columns that provide atom mapping information are dropped
atom_columns_to_drop = ["C1", "O2", "O3", "C4", "H5"]

#title of the column for the energy you want to use for boltzmann averaging and lowest E conformer determination
energy_col_header = "G(T)_spc(Hartree)"


energy_cutoff = 4.2 #specify energy cutoff in kcal/mol to remove conformers above this value before post-processing
verbose = False #set to true if you'd like to see info on the nunmber of conformers removed for each compound

### Option to import an Excel sheet if you're using properties or energies collected outside of this notebook

If you would like to use post-processing functionality (i.e. Boltzmann averaging, lowest E conformers, etc.) you can read in a dataframe with properties (e.g. QikProp properties) or energies (e.g. if you don't/can't run linked jobs) collected outside of this notebook. 

Check out the dataframe_sample.xlsx to make sure you have the desired format. 

In [ ]:
df = pd.read_excel('All_Conformer_Properties_example.xlsx','Sheet1',index_col=0,header=0,engine='openpyxl')
display(df.head())

## Generating a list of compounds that have conformational ensembles

**ONLY RUN THE AUTOMATED OR THE MANUAL CELL, NOT BOTH**

**AUTOMATED:** if your compounds are named consistenly, this section generates your compound list based on the similar naming structure

In [10]:
#this is a template for our files that are all named "AcXXX_clust-X.log"

compound_list = []
    
for index, row in df.iterrows():
    log_file = row['log_name'] #read file name from df
    prefix_and_compound = log_file.split(str(suffix)) #splits to get "AcXXX" (entry O) (and we don't use the "clust-X" (entry 1))
    compound = prefix_and_compound[0].split(str(prefix)) #splits again to get "XXX" (entry 1) (and we don't use the empty string "" (entry 0))
    compound_list.append(compound[1])

compound_list = list(set(compound_list)) #removes duplicate stuctures that result from having conformers of each
compound_list.sort() #reorders numerically (not sure if it reorders alphabetically)
print(compound_list)

#this should generate a list that looks like this: ['24', '27', '34', '48']

['1', '2', '3', '4', '5']


**MANUAL:** if your comment naming scheme is not consistent or you have trouble with the template above, you can manually define your compound list

In [ ]:
compound_list = [1, 2, 3, 4, 5]

## Post-processing to get properties for each compound

In [11]:
all_df_master = pd.DataFrame(columns=[])
properties_df_master = pd.DataFrame(columns=[])

for compound in compound_list: 
    #defines the common start to all files using the input above 
    substring = str(prefix) + str(compound) + str(suffix)
    
    #makes a data frame for one compound at a time for post-processing
    valuesdf = df[df["log_name"].str.startswith(substring)]
    valuesdf = valuesdf.drop(columns = atom_columns_to_drop)
    valuesdf = valuesdf.reset_index(drop = True)  #you must re-index otherwise the 2nd, 3rd, etc. compounds fail
   
    #define columns that won't be included in summary properties or are treated differently because they don't make sense to Boltzmann average
    non_boltz_columns = ["G(Hartree)","∆G(Hartree)","∆G(kcal/mol)", "e^(-∆G/RT)","Mole Fraction"] #don't boltzman average columns containing these strings in the column label
    reg_avg_columns = ['CPU_time_total(hours)', 'Wall_time_total(hours)'] #don't boltzmann average these either, we average them in case that is helpful
    gv_extra_columns = ['E_spc (Hartree)', 'H_spc(Hartree)', 'T', 'T*S', 'T*qh_S', 'ZPE(Hartree)', 'qh_G(T)_spc(Hartree)', "G(T)_spc(Hartree)"]
    gv_extra_columns.remove(str(energy_col_header))
    
    #calculate the summary properties based on all conformers (Boltzmann Average, Minimum, Maximum, Boltzmann Weighted Std)
    valuesdf["∆G(Hartree)"] = valuesdf[energy_col_header] - valuesdf[energy_col_header].min()
    valuesdf["∆G(kcal/mol)"] = valuesdf["∆G(Hartree)"] * 627.5
    valuesdf["e^(-∆G/RT)"] = np.exp((valuesdf["∆G(kcal/mol)"] * -1000) / (1.987204 * 298.15)) #R is in cal/(K*mol)
    valuesdf["Mole Fraction"] = valuesdf["e^(-∆G/RT)"] / valuesdf["e^(-∆G/RT)"].sum()
    initial = len(valuesdf.index)
    if verbose: 
        print(prefix + str(compound))
        #display(valuesdf)
        print("Total number of conformers = ", initial)
    valuesdf.drop(valuesdf[valuesdf["∆G(kcal/mol)"] >= energy_cutoff].index, inplace=True) #E cutoff applied here
    valuesdf = valuesdf.reset_index(drop = True) #resetting indexes
    final = len(valuesdf.index) 
    if verbose: 
        print("Number of conformers above ", energy_cutoff, " kcal/mol: ", initial-final)
    values_boltz_row = []
    values_min_row = []
    values_max_row = []
    values_boltz_stdev_row =[]
    values_range_row = []
    values_exclude_columns = []
    
    for column in valuesdf:
        if "log_name" in column:
            values_boltz_row.append("Boltzmann Averages")
            values_min_row.append("Ensemble Minimum")
            values_max_row.append("Ensemble Maximum")
            values_boltz_stdev_row.append("Boltzmann Standard Deviation")
            values_range_row.append("Ensemble Range")
            values_exclude_columns.append(column) #used later to build final dataframe
        elif any(phrase in column for phrase in non_boltz_columns) or any(phrase in column for phrase in gv_extra_columns):
            values_boltz_row.append("")
            values_min_row.append("")
            values_max_row.append("")
            values_boltz_stdev_row.append("")
            values_range_row.append("")
        elif any(phrase in column for phrase in reg_avg_columns):
            values_boltz_row.append(valuesdf[column].mean()) #intended to print the average CPU/wall time in the boltz column
            values_min_row.append("")
            values_max_row.append("")
            values_boltz_stdev_row.append("")
            values_range_row.append("")
        else:
            valuesdf[column] = pd.to_numeric(valuesdf[column]) #to hopefully solve the error that sometimes occurs where the float(Mole Fraction) cannot be mulitplied by the string(property)
            values_boltz_row.append((valuesdf[column] * valuesdf["Mole Fraction"]).sum())
            values_min_row.append(valuesdf[column].min())
            values_max_row.append(valuesdf[column].max())
            values_range_row.append(valuesdf[column].max() - valuesdf[column].min())

            
            # this section generates the weighted std deviation (weighted by mole fraction) 
            # formula: https://www.statology.org/weighted-standard-deviation-excel/
    
            boltz = (valuesdf[column] * valuesdf["Mole Fraction"]).sum() #number
            delta_values_sq = []
    
            #makes a list of the "deviation" for each conformer           
            for index, row in valuesdf.iterrows(): 
                value = row[column]
                delta_value_sq = (value - boltz)**2
                delta_values_sq.append(delta_value_sq)
            
            #w is list of weights (i.e. mole fractions)
            w = list(valuesdf["Mole Fraction"])
            wstdev = np.sqrt( (np.average(delta_values_sq, weights=w)) / (((len(w)-1)/len(w))*np.sum(w)) )
            if len(w) == 1: #if there is only one conformer in the ensemble, set the weighted standard deviation to 0 
                wstdev = 0
            #np.average(delta_values_sq, weights=w) generates sum of each (delta_value_sq * mole fraction)
            
            values_boltz_stdev_row.append(wstdev)
            
            
    valuesdf.loc[len(valuesdf)] = values_boltz_row
    valuesdf.loc[len(valuesdf)] = values_boltz_stdev_row
    valuesdf.loc[len(valuesdf)] = values_min_row
    valuesdf.loc[len(valuesdf)] = values_max_row
    valuesdf.loc[len(valuesdf)] = values_range_row

    #final output format is built here:
    explicit_order_front_columns = ["log_name", energy_col_header,"∆G(Hartree)","∆G(kcal/mol)","e^(-∆G/RT)","Mole Fraction"]
    
    #reorders the dataframe using front columns defined above
    valuesdf = valuesdf[explicit_order_front_columns + [col for col in valuesdf.columns if col not in explicit_order_front_columns and col not in values_exclude_columns]]
    
    #determine the index of the lowest energy conformer
    low_e_index = valuesdf[valuesdf["∆G(Hartree)"] == 0].index.tolist()
    
    #copy the row to a new_row with the name of the log changed to Lowest E Conformer
    new_row = valuesdf.loc[low_e_index[0]]
    new_row['log_name'] = "Lowest E Conformer"   
    valuesdf =  valuesdf.append(new_row, ignore_index=True)

#------------------------------EDIT THIS SECTION IF YOU WANT A SPECIFIC CONFORMER----------------------------------  
    #if you want all properties for a conformer with a particular property (i.e. all properties for the Vbur_min conformer)
    #this template can be adjusted for min/max/etc. 
    
    #find the index for the min or max column:
    ensemble_min_index = valuesdf[valuesdf["log_name"] == "Ensemble Minimum"].index.tolist()
    
    #find the min or max value of the property (based on index above)
    #saves the value in a list (min_value) with one entry (this is why we call min_value[0])
    min_value = valuesdf.loc[ensemble_min_index, "%Vbur_C4_3.0Å"].tolist()   
    vbur_min_index = valuesdf[valuesdf["%Vbur_C4_3.0Å"] == min_value[0]].index.tolist()
    
    #copy the row to a new_row with the name of the log changed to Property_min_conformer
    new_row = valuesdf.loc[vbur_min_index[0]]
    new_row['log_name'] = "%Vbur_C4_3.0Å_min_Conformer"   
    valuesdf =  valuesdf.append(new_row, ignore_index=True)
#--------------------------------------------------------------------------------------------------------------------    
    
    #appends the frame to the master output
    all_df_master = pd.concat([all_df_master, valuesdf])
    
    #drop all the individual conformers
    dropindex = valuesdf[valuesdf["log_name"].str.startswith(substring)].index
    valuesdf = valuesdf.drop(dropindex)
    valuesdf = valuesdf.reset_index(drop = True)
    
    #display(valuesdf)   
    
    #drop the columns created to determine the mole fraction and some that 
    valuesdf = valuesdf.drop(columns = explicit_order_front_columns)
    try:
        valuesdf = valuesdf.drop(columns = gv_extra_columns)
    except:
        pass
    try:
        valuesdf = valuesdf.drop(columns = reg_avg_columns)
    except:
        pass
        
#---------------------THIS MAY NEED TO CHANGE DEPENDING ON HOW YOU LABEL YOUR COMPOUNDS------------------------------  
    compound_name = prefix + str(compound) 
#--------------------------------------------------------------------------------------------------------------------      

    properties_df = pd.DataFrame({'Compound_Name': [compound_name]})
    
    #builds a dataframe (for each compound) by adding summary properties as new columns
    for (columnName, columnData) in valuesdf.iteritems():
        #the indexes need to match the values dataframe - display it to double check if you need to make changes 
        #(uncomment the display(valuesdf) in row 124 of this cell)
        properties_df[str(columnName) + "_Boltz"] = [columnData.values[0]]
        properties_df[str(columnName) + "_Boltz_stdev"] = [columnData.values[1]]
        properties_df[str(columnName) + "_min"] = [columnData.values[2]]
        properties_df[str(columnName) + "_max"] = [columnData.values[3]]
        properties_df[str(columnName) + "_range"] = [columnData.values[4]]
        properties_df[str(columnName) + "_low_E"] = [columnData.values[5]]
        
        #if you're collecting properties for a specific conformer, add these here (note the index)
        #example:
        properties_df[str(columnName) + "_V_bur_min"] = [columnData.values[6]]
        
        #if you only want a table with Boltz, you can comment out the other summary properties to generate a Boltz spreadsheet
        #of if you don't want to collect range, etc.
    #concatenates the individual acid properties df into the master properties df
    properties_df_master = pd.concat([properties_df_master, properties_df], axis = 0)

all_df_master = all_df_master.reset_index(drop = True)
properties_df_master = properties_df_master.reset_index(drop = True)


### Peek at your new dataframes

In [12]:
display(properties_df_master.head())
display(all_df_master)

,Compound_Name,HOMO_Boltz,HOMO_Boltz_stdev,HOMO_min,HOMO_max,HOMO_range,HOMO_low_E,HOMO_V_bur_min,LUMO_Boltz,LUMO_Boltz_stdev,LUMO_min,LUMO_max,LUMO_range,LUMO_low_E,LUMO_V_bur_min,η_Boltz,η_Boltz_stdev,η_min,η_max,η_range,η_low_E,η_V_bur_min,μ_Boltz,μ_Boltz_stdev,μ_min,μ_max,μ_range,μ_low_E,μ_V_bur_min,ω_Boltz,ω_Boltz_stdev,ω_min,ω_max,ω_range,ω_low_E,ω_V_bur_min,%Vbur_C1_2.0Å_Boltz,%Vbur_C1_2.0Å_Boltz_stdev,%Vbur_C1_2.0Å_min,%Vbur_C1_2.0Å_max,%Vbur_C1_2.0Å_range,%Vbur_C1_2.0Å_low_E,%Vbur_C1_2.0Å_V_bur_min,%Vbur_C4_2.0Å_Boltz,%Vbur_C4_2.0Å_Boltz_stdev,%Vbur_C4_2.0Å_min,%Vbur_C4_2.0Å_max,%Vbur_C4_2.0Å_range,%Vbur_C4_2.0Å_low_E,%Vbur_C4_2.0Å_V_bur_min,%Vbur_C1_2.5Å_Boltz,%Vbur_C1_2.5Å_Boltz_stdev,%Vbur_C1_2.5Å_min,%Vbur_C1_2.5Å_max,%Vbur_C1_2.5Å_range,%Vbur_C1_2.5Å_low_E,%Vbur_C1_2.5Å_V_bur_min,%Vbur_C4_2.5Å_Boltz,%Vbur_C4_2.5Å_Boltz_stdev,%Vbur_C4_2.5Å_min,%Vbur_C4_2.5Å_max,%Vbur_C4_2.5Å_range,%Vbur_C4_2.5Å_low_E,%Vbur_C4_2.5Å_V_bur_min,%Vbur_C1_3.0Å_Boltz,%Vbur_C1_3.0Å_Boltz_stdev,%Vbur_C1_3.0Å_min,%Vbur_C1_3.0Å_max,%Vbur_C1_3.0Å_range,%Vbur_C1_3.0Å_low_E,%Vbur_C1_3.0Å_V_bur_min,%Vbur_C4_3.0Å_Boltz,%Vbur_C4_3.0Å_Boltz_stdev,%Vbur_C4_3.0Å_min,%Vbur_C4_3.0Å_max,%Vbur_C4_3.0Å_range,%Vbur_C4_3.0Å_low_E,%Vbur_C4_3.0Å_V_bur_min,%Vbur_C1_3.5Å_Boltz,%Vbur_C1_3.5Å_Boltz_stdev,%Vbur_C1_3.5Å_min,%Vbur_C1_3.5Å_max,%Vbur_C1_3.5Å_range,%Vbur_C1_3.5Å_low_E,%Vbur_C1_3.5Å_V_bur_min,%Vbur_C4_3.5Å_Boltz,%Vbur_C4_3.5Å_Boltz_stdev,%Vbur_C4_3.5Å_min,%Vbur_C4_3.5Å_max,%Vbur_C4_3.5Å_range,%Vbur_C4_3.5Å_low_E,%Vbur_C4_3.5Å_V_bur_min,%Vbur_C1_4.0Å_Boltz,%Vbur_C1_4.0Å_Boltz_stdev,%Vbur_C1_4.0Å_min,%Vbur_C1_4.0Å_max,%Vbur_C1_4.0Å_range,%Vbur_C1_4.0Å_low_E,%Vbur_C1_4.0Å_V_bur_min,%Vbur_C4_4.0Å_Boltz,%Vbur_C4_4.0Å_Boltz_stdev,%Vbur_C4_4.0Å_min,%Vbur_C4_4.0Å_max,%Vbur_C4_4.0Å_range,%Vbur_C4_4.0Å_low_E,%Vbur_C4_4.0Å_V_bur_min
0,Ac1,-0.355157,0.001852,-0.35996,-0.35118,0.00878,-0.35441,-0.35482,0.043565,0.001970,0.03908,0.04621,0.00713,0.04263,0.04101,0.398722,0.003274,0.39145,0.40570,0.01425,0.39704,0.39583,-0.155796,0.000988,-0.158315,-0.153955,0.00436,-0.155890,-0.156905,0.030440,0.000476,0.02982,0.03164,0.00182,0.03060,0.03110,94.307963,0.676781,93.914644,96.600594,2.685950,94.266529,96.064695,97.531890,2.806115e-01,97.136493,98.573089,1.436596,97.491606,97.136493,83.169595,1.237045,82.071233,87.604180,5.532947,83.596497,85.509181,83.808537,0.651713,82.894908,87.278617,4.383709,83.428832,82.992577,66.891028,1.772922,65.156622,72.508194,7.351572,67.709885,69.311494,68.451059,1.014143,67.682881,73.606973,5.924091,67.843973,67.682881,50.173751,2.144430,48.030423,56.595928,8.565505,51.158335,52.907193,55.676206,1.275044,54.914624,61.602567,6.687943,54.993245,54.992080,37.789800,2.193347,35.613090,44.416082,8.802992,38.623844,40.855627,43.728287,1.340524,42.991978,49.866163,6.874185,43.012150,43.393489
1,Ac2,-0.331555,0.001875,-0.33516,-0.32750,0.00766,-0.33121,-0.32995,0.031340,0.003806,0.02678,0.03818,0.01140,0.02970,0.03761,0.362895,0.005322,0.35428,0.37334,0.01906,0.36091,0.36756,-0.150108,0.001386,-0.150755,-0.146165,0.00459,-0.150755,-0.146170,0.031062,0.000945,0.02906,0.03191,0.00285,0.03149,0.02906,93.936250,0.086817,93.892045,95.354468,1.462423,93.924329,93.892045,97.199023,2.607002e-01,96.600594,97.756327,1.155733,97.281767,96.600594,81.749530,0.539012,81.517776,85.053392,3.535617,81.517776,81.840083,82.550759,0.188490,82.110301,83.143964,1.033663,82.531905,82.113556,64.790530,1.091550,64.296223,70.949419,6.653196,64.296223,65.680870,66.295835,0.193098,66.148316,66.801065,0.652749,66.212567,66.148316,47.692421,1.659719,46.763185,56.094507,9.331322,46.937314,49.474119,53.321994,0.275116,53.219926,54.673523,1.453597,53.226332,53.219926,35.332161,1.929149,34.296055,44.765223,10.469167,34.445022,37.594656,42.169106,0.565738,41.993436,44.751645,2.758209,41.993436,42.083049
2,Ac3,-0.361880,0.000000,-0.36188,-0.36188,0.00000,-0.36188,-0.36188,-0.060890,0.000000,-0.06089,-0.06089,0.00000,-0.06089,-0.06089,0.300990,0.000000,0.30099,0.30099,0.00000,0.30099,0.

,log_name,G(T)_spc(Hartree),∆G(Hartree),∆G(kcal/mol),e^(-∆G/RT),Mole Fraction,E_spc (Hartree),H_spc(Hartree),T,T*S,T*qh_S,ZPE(Hartree),qh_G(T)_spc(Hartree),HOMO,LUMO,η,μ,ω,%Vbur_C1_2.0Å,%Vbur_C4_2.0Å,%Vbur_C1_2.5Å,%Vbur_C4_2.5Å,%Vbur_C1_3.0Å,%Vbur_C4_3.0Å,%Vbur_C1_3.5Å,%Vbur_C4_3.5Å,%Vbur_C1_4.0Å,%Vbur_C4_4.0Å
0,Ac1_clust-10,-543.292382,0.006642,4.167876,0.000881,0.000275,-543.537866,-543.230502,298.15,0.06188,0.057455,0.291621,-543.287958,-0.35342,0.04394,0.39736,-0.154740,0.03013,94.473140,97.536803,82.479815,85.004558,65.471357,71.097475,48.637835,59.546217,37.247455,48.395118
1,Ac1_clust-12,-543.294717,0.004308,2.703041,0.010439,0.003258,-543.540547,-543.233299,298.15,0.061417,0.05729,0.291455,-543.290589,-0.35482,0.04101,0.39583,-0.156905,0.03110,96.064695,97.136493,85.509181,82.992577,69.311494,67.682881,52.907193,54.992080,40.855627,43.393489
2,Ac1_clust-18,-543.294049,0.004975,3.122097,0.005146,0.001606,-543.542172,-543.234779,298.15,0.059269,0.055892,0.291974,-543.290671,-0.35763,0.04185,0.39948,-0.157890,0.03120,96.600594,97.427040,87.545579,83.977406,72.267022,69.966105,56.176039,58.693044,43.969958,47.363990
3,Ac1_clust-19,-543.29505,0.003974,2.493998,0.014855,0.004637,-543.542868,-543.235277,298.15,0.059773,0.056092,0.292182,-543.291369,-0.35484,0.03948,0.39432,-0.157680,0.03153,96.148631,97.243027,86.515171,84.332270,71.450387,70.174687,55.879612,58.273737,44.008364,46.574545
4,Ac1_clust-2,-543.294733,0.004291,2.692864,0.01062,0.003315,-543.542563,-543.235274,298.15,0.059459,0.056186,0.291754,-543.29146,-0.35712,0.04151,0.39863,-0.157805,0.03124,96.529571,97.420584,87.604180,84.088097,72.508194,69.796633,56.595928,57.907426,44.416082,46.173810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,Ensemble Minimum,,,,,,,,,,,,,-0.30391,0.00976,0.31367,-0.147075,0.03086,95.645015,99.970945,87.426748,98.504037,75.021417,91.903494,61.682352,78.593808,51.141689,60.729083
80,Ensemble Maximum,,,,,,,,,,,,,-0.29989,0.01922,0.31911,-0.140335,0.03448,96.264850,99.970945,88.947129,98.814950,76.246834,92.470575,62.492429,78.995061,51.656089,61.025852
81,Ensemble Range,,,,,,,,,,,,,0.00402,0.00946,0.00544,0.006740,0.00362,0.619835,0.000000,1.520380,0.310913,1.225417,0.567081,0.810077,0.401253,0.514400,0.296769
82,Lowest E Conformer,-538.575959,0.0,0.0,1.0,0.819899,-538.7393,-538.526495,298.15,0.049465,0.048124,0.200661,-538.574618,-0.30375,0.01260,0.31635,-0.145575,0.03349,96.264850,99.970945,88.947129,98.814950,76.246834,92.470575,62.492429,78.988655,51.656089,60.988998


### Save to Microsoft Excelᵀᴹ 

In [13]:
all_df_master.to_excel('All_Conformer_and_Summary_Properties_example.xlsx', index = False)
properties_df_master.to_excel('Summary_Properties_example.xlsx', index = False)